In [ ]:
# Packages that we are going to use in this project
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


import sys 
import os

import matplotlib.pyplot as plt
import plotly.express as px

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf

from scipy.fftpack import fft
from scipy.signal import blackman
from scipy.signal import periodogram

from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
from sklearn.preprocessing import MinMaxScaler

from functions import *


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\df")
from manage_file import FILES, getPath



## STEP 1: Read & view of the data

In [ ]:
# READ DATA INPUT : earth, moon or minable. 
df = read_data(file="earth")

In [ ]:
# plot map : recieve dataframe , cluster = Boolean and specific cluster 
plot_map(df)

## STEP 2:  __Frequency for a period of time of earthquakes considering magnitude segmentation.__

For ex:
* period_length = 10 (every 10 years compare)
* mag_seg = {magnitude < 4 |  4<= magnitude < 5 | 5 < magnitude }

In [ ]:
######## you must want to change these piece of code ########
period_length = 10
#############################################################
df = period_calculation(df, period_length)# how many earthquakes for each period

In [ ]:
######## you must want to change these piece of code ########
mag_seg = {0: [0,4.6], 1: [4.6,4.9], 2: [4.9,10]} 
#############################################################
df = magnitude_segmentation(df, mag_seg)

### Clustering

In [ ]:
df = calculate_clustering(df, normalized=False)

# STAGE 2:  Relation between moon and eartquakes

In [ ]:
# READ DATA INPUT : earth, moon or minable_{period_len}
df_moon = read_data(file="moon")

In [ ]:
df_merged = pd.merge(df, df_moon, left_on=['year', 'month', 'day'], right_on=['year', 'month', 'day'], how="left")

In [ ]:
var_name = ["ill_frac","r/km", "dec"] # variable to interpolate
print(df_moon[var_name].describe())
#The next line will apply the interpolation. This line will take time. If is not neccesary reapply dont run it
# remember that minable has interpolation already done.
df_merged = apply_interpolation(df_merged, df_moon, var_name) 

In [ ]:
# save the dataframe 
df_merged.to_csv(getPath(FILES.minable))

## ADDED :: Tidal Acceleration


In [ ]:
def particionar(row):
    columnas = row.split(" ")
    columnas = [c for c in columnas if c!= ""]
    for i in range(len(columnas)):
        try:
            columnas[i] = float(columnas[i])
        except:
            pass
    return columnas
def create_tidal_demo():
    ano = 1973
    info = []
    for i in range(len(df_td)):
        if not pd.isna(df_td.iloc[i,8]):
            ano = int(df_td.iloc[i,8])
        if (not pd.isna(df_td.iloc[i,0])):
            if (df_td.iloc[i,0] == "D E M O"):
                r = [ano,info[-1][1], info[-1][2]+1, " ", " ", " ", " ", " ", " ", " ", " "]
            else:
                r = particionar(df_td.iloc[i,0])
                r.insert(0, ano)
            info.append(r)
    df_tidal = pd.DataFrame(info, columns = ["year","month","day","r/km", "arcmin", "phase", "dist/km", "arcmin2", "td_acc", "dist/au", "F(EM)/F(SM)"])
    
    
meses_abreviados = {
    "jan": 1,
    "feb": 2,
    "mar": 3,
    "apr": 4,
    "may": 5,
    "jun": 6,
    "jul": 7,
    "aug": 8,
    "sep": 9,
    "oct": 10,
    "nov": 11,
    "dec": 12
}
df_td = df_td[['year', 'month', 'day', 'td_acc']]
df_td["month"] = df_td["month"].apply(lambda x: meses_abreviados.get(x.lower(), None))